<a href="https://colab.research.google.com/github/keerthi09090/CSCI-167/blob/main/Notebook_12_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [20]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [21]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [22]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [23]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
    # Compute the keys, queries and values.
    query = np.dot(omega_q, x) + beta_q
    key = np.dot(omega_k, x) + beta_k
    value = np.dot(omega_v, x) + beta_v
    all_queries.append(query)
    all_keys.append(key)
    all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [24]:
def softmax(items_in):
    # Compute the elements of items_out
    exp_items = np.exp(items_in - np.max(items_in))
    items_out = exp_items / np.sum(exp_items)
    return items_out

Now compute the self attention values:

In [25]:
# Create empty list for output
all_x_prime = []
# For each output
for n in range(N):
    # Create list for dot products of query N with all keys
    all_km_qn = []
    # Compute the dot products
    for key in all_keys:
        # Compute the appropriate dot product
        dot_product = np.dot(all_queries[n].T, key)
        all_km_qn.append(dot_product)
    # Compute dot product
    attention = softmax(all_km_qn)
    # Print result (should be positive sum to one)
    print("Attentions for output ", n)
    print(attention)
    # Compute a weighted sum of all of the values according to the attention (equation 12.3)
    x_prime = sum(attention[i] * all_values[i] for i in range(len(all_keys)))
    all_x_prime.append(x_prime)

# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004 4.02764044 2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533 3.96863308 2.15858316]]")


Attentions for output  0
[[[1.24326146e-13]]

 [[9.98281489e-01]]

 [[1.71851130e-03]]]
Attentions for output  1
[[[2.79525306e-12]]

 [[5.85506360e-03]]

 [[9.94144936e-01]]]
Attentions for output  2
[[[0.00505708]]

 [[0.00654776]]

 [[0.98839516]]]
x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_1_true: [[ 1.64201168 -0.08470004 4.02764044 2.18690791]]
x_prime_2_calculated: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]
x_prime_2_true: [[ 1.61949281 -0.06641533 3.96863308 2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [26]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [27]:
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    # 1. Compute queries, keys, and values
    queries = np.dot(omega_q, X) + beta_q  # Shape: (D, N)
    keys = np.dot(omega_k, X) + beta_k     # Shape: (D, N)
    values = np.dot(omega_v, X) + beta_v   # Shape: (D, N)

    # 2. Compute dot products between queries and keys
    # Shape: (N, N)
    dot_products = np.dot(queries.T, keys)

    # 3. Apply softmax to calculate attentions
    # Shape: (N, N)
    attentions = softmax_cols(dot_products.T)

    # 4. Weight values by attentions
    # Final shape: (D, N)
    X_prime = np.dot(values, attentions)

    return X_prime

# Run the self attention mechanism
X_prime = self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

In [28]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.94744244  1.64201168  1.61949281]
 [-0.24348429 -0.08470004 -0.06641533]
 [-0.91310441  4.02764044  3.96863308]
 [-0.44522983  2.18690791  2.15858316]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [29]:
def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    # 1. Compute queries, keys, and values
    queries = np.dot(omega_q, X) + beta_q  # Shape: (D, N)
    keys = np.dot(omega_k, X) + beta_k     # Shape: (D, N)
    values = np.dot(omega_v, X) + beta_v   # Shape: (D, N)

    # 2. Compute dot products between queries and keys
    # Shape: (N, N)
    dot_products = np.dot(queries.T, keys)

    # 3. Scale the dot products by sqrt(D) as in equation 12.9
    D = queries.shape[0]  # Get dimension D
    scaled_dot_products = dot_products / np.sqrt(D)

    # 4. Apply softmax to calculate attentions
    # Shape: (N, N)
    attentions = softmax_cols(scaled_dot_products)

    # 5. Weight values by attentions
    # Final shape: (D, N)
    X_prime = np.dot(values, attentions)

    return X_prime

In [30]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 1.64608331  0.66149223  1.21854105]
 [-0.08376846  0.36015254 -0.17896151]
 [ 4.05660686 -0.05852563  1.02632615]
 [ 2.20233961  0.09885586  0.58832268]]


TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.

The spreadsheet and Jupyter Notebook both explain self-attention, a key mechanism in machine learning, but differ in presentation. Both cover core concepts like queries, keys, values, and attention weights, and provide step-by-step calculations to clarify the process.

The notebook is interactive, allowing users to modify parameters, run code, and visualize results, making it ideal for hands-on learners. It also includes detailed explanations and dynamic plots. In contrast, the spreadsheet is static, formula-driven, and focuses on a fixed example with less theoretical depth, suited for users who prefer structured, step-by-step calculations without coding.

In summary, the notebook offers a flexible, interactive learning experience, while the spreadsheet provides a more structured, formulaic approach, catering to different learning preferences.
